# Parking Usage in the City of Cambridge

*The focus of this analysis will be the length of stay and type of user of a parking lot in the city of Cambridge. The available data starts on 01/04/2015 and goes until 31/12/2022.*

*Source of the data: https://ckan.publishing.service.gov.uk/dataset/cambridge-city-car-park-usage-data-by-length-of-stay*

### Parking lots in the scope of this analysis:

- Grafton East (GE)
    

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, chi2_contingency
import datetime

In [ ]:
# To make the output look nicer
np.set_printoptions(suppress=True, precision = 2)

## 1 - Data Loading and Checking

In [ ]:
# Load the Grafton East (GE) parking dataset
ge_data = pd.read_csv('data/grafton_east.csv')

# Inspect the first 10 rows
ge_data.head(10)

## 2 - Data Wrangling and Tidyng

In [ ]:
# In order to have a better understanting of the data
ge_data.info()

In [ ]:
# In order to dentify the continuous and categorical columns in the data
ge_data.nunique()

#### Based on this preliminary analysis, we can make some assumptions about the data:

- A lot of columns appear to have extra spaces before and after their names
- The columns `Up to 1 hr`, `1 to 2 hrs`, `2 to 3 hrs`, `3 to 4 hrs`, `4 to 5 hrs`, `5 to 6 hrs`, `6 to <24 hours`, `24 hours +`, `Total Exc Sub` and `Subscribers` all look as int64 variables, but only the `Total Exc Sub` is reflected as one.

- The column `Comments`, despite the int values should not be one. It's better to replace the values for a better fit. The columns also have other factors to be checked.

- It's a good idea to convert the `Date`column into a `datetime` format, this way it will be easy to perform operations later

- It is also a good idea to convert the `Day` of the week into a category variable, so we can organize the week starting on Monday and ending on Sunday.

In [ ]:
# Set Day variable as categorical
ge_data['Day'] = pd.Categorical(ge_data['Day'], 
                      categories=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
                      ordered=True)

ge_data['Day'].dtypes

In [ ]:
# Converting the dd/mm/yyyy into yyyy-mm-dd format
ge_data['Date'] = pd.to_datetime(ge_data['Date'], dayfirst=True, format="%d/%m/%Y")

In [ ]:
# Function to remove extra spaces from the entire data 
def whitespace_remover(df):
    
    # first to remove the extra spaces in all columns
    ge_data.columns = ge_data.columns.str.strip()
    
    # iterating over the columns to remove spaces from values
    for i in df.columns:
 
        # checking datatype of each columns
        if df[i].dtype == 'object':
 
            # applying strip function
            df[i] = df[i].str.strip()
        
        else:
            pass
        
    return df

In [ ]:
# Applying the function to remove spaces in the dataframe
whitespace_remover(ge_data)

**Let's look at the unique values of the variables we think should be int64 to understand why they are reflected as objects by python:**

Starting with `Up to 1 hr`:

In [ ]:
ge_data['Up to 1 hr'].unique()

Looking at the unique values it's possible to see some '-' in the middle of the values and also a value of '1,678' that looks like a typo with an extra digit. **Let's take a closer look at those cases:**

In [ ]:
# First lets check the rows with the '-' value for the column 'Up to 1 hr'
ge_data.loc[ge_data['Up to 1 hr'] == '-']

In [ ]:
# Now lets check the rows with the '1,678' value for the column 'Up to 1 hr'
ge_data.loc[ge_data['Up to 1 hr'] == '1,678']

It appears we were correct and the value '-' would be better represented by 0 and the value '1.678' actually appears to be a typo. **Lets replace that '-' with 0, that '1,678' with 167 and see if we can change the variable to int64:**

In [ ]:
ge_data['Up to 1 hr'] = ge_data['Up to 1 hr'].replace({'-': 0, '1,678': 167})
ge_data['Up to 1 hr'] = ge_data['Up to 1 hr'].astype('int64')

In [ ]:
# Let's check if the change into int64 is done
ge_data.info()

We did it! During the process it was possible to notice that this '-' value appears throughout the entire dataframe, **so we will apply the replacement with 0 throughout the dataframe:**

In [ ]:
# Replaces '-' with 0 in the entire dataframe
ge_data = ge_data.replace('-', 0)

**Now let's apply the same logic to convert the other variables to int:**

In [ ]:
ge_data['1 to 2 hrs'] = ge_data['1 to 2 hrs'].astype('int64')
ge_data['2 to 3 hrs'] = ge_data['2 to 3 hrs'].astype('int64')
ge_data['3 to 4 hrs'] = ge_data['3 to 4 hrs'].astype('int64')
ge_data['4 to 5 hrs'] = ge_data['4 to 5 hrs'].astype('int64')
ge_data['5 to 6 hrs'] = ge_data['5 to 6 hrs'].astype('int64')
ge_data['6 to <24 hours'] = ge_data['6 to <24 hours'].astype('int64')
ge_data['24 hours +'] = ge_data['24 hours +'].astype('int64')
ge_data['Subscribers'] = ge_data['Subscribers'].astype('int64')
ge_data['Total Exc Sub'] = ge_data['Total Exc Sub'].astype('int64')

In [ ]:
# Let's check again if the changes into int64 are done
ge_data.info()

**Excellent!** We're almost there, now we need to check the `Comments` column, which despite being an object, apparently has 0 and other values. Let's take a look at the unique values:

In [ ]:
# Check the unique values of the Comments variable
ge_data['Comments'].unique()

In [ ]:
# Check the rows with '0' as the value for the Comments variable
ge_data.loc[ge_data['Comments'] == '0']

Based on this, it is much more interesting to **change the values from '0' to** `null` and thus have more consistency in the values:

In [ ]:
# Replacing the value og '0' for null
ge_data['Comments'] = ge_data['Comments'].replace('0', np.nan)

# Checking the changes
ge_data['Comments'].unique()

Now let's use the `describe()` method to see if we have anything else that can catch our attention with all the variables properly corrected.

In [ ]:
ge_data.describe()

**Weird!** It doesn't make sense to have a negative number of `Total Exc Sub`, so **lets first check that row:**

In [ ]:
ge_data.loc[ge_data['Total Exc Sub'] == -1]

It seems that the negative number is a consequence of subtracting stays from the number of subscribers, it is difficult to be sure where the error is. As it is something that will not affect the data analysis, I will change the value to 0. 

We could just use a `replace` method, but I will use `lambda` because it could be used to apply other iterations to the values of a column.

In [ ]:
# Using lambda to iterate over all values and replace negative values with 0
ge_data['Total Exc Sub'] = ge_data['Total Exc Sub'].apply(lambda x: 0 if x < 0 else x)

## 2 - Data Exploration

**We finally arrived** at the version of our dataframe after wrangling and tidyng. To start lets look at some summary statistics for all features in the dataset:

In [ ]:
ge_data.describe()

Analyzing the information it is possible to see that on average cars spend between 1 and 3 hours in the parking lot most frequently, with between 1 and 2 hours being the longest period on average.

**From the data it is possible to see that there are 2 types of users: `Subscribers` and `Non-Subscribers` let's compare these 2 user profiles during the days of the week using a `boxplot` and see if we have any relationship between them:**

In [ ]:
sns.boxplot(data = ge_data, x = 'Day', y = 'Total Exc Sub')
plt.show()

In [ ]:
sns.boxplot(data = ge_data, x = 'Day', y = 'Subscribers')
plt.show()

**Analyzing the graphs we can see patterns for the 2 groups:**
- `Non-Subscribers`: These are users who use parking in a more random and non-sequential manner. We see a large increase in the presence of these users on the weekend, probably using parking for pleasure.
- `Subscribers`: These are users who use the parking lot frequently, probably because they park their car to go to work. This hypothesis is reinforced by the significant decrease in the number of subscribers present at the weekend.

## FOR FUTURE IMPLEMENTATION

In [ ]:
def analyse_year(data, year):
    ...

In [ ]:
def compare_between_years(data, year_1, year_2):
    ...

For a deeper analysis, we will **divide the dataframe by years** and use only the last 5, namely: `2018, 2019, 2020, 2021 and 2022`:

In [ ]:
ge2018_data = ge_data.loc[(ge_data['Date'] >= '2018-01-01') & (ge_data['Date'] <= '2018-12-31')]
ge2019_data = ge_data.loc[(ge_data['Date'] >= '2019-01-01') & (ge_data['Date'] <= '2019-12-31')]
ge2020_data = ge_data.loc[(ge_data['Date'] >= '2020-01-01') & (ge_data['Date'] <= '2020-12-31')]
ge2021_data = ge_data.loc[(ge_data['Date'] >= '2021-01-01') & (ge_data['Date'] <= '2021-12-31')]
ge2022_data = ge_data.loc[(ge_data['Date'] >= '2022-01-01') & (ge_data['Date'] <= '2022-12-31')]